# Parsing Tabular Data

**Originally Contributed by**: Clayton Barrows

## Introduction

An example of how to parse tabular files (CSV) files similar to the format established in
the [RTS-GMLC](github.com/gridmod/rts-gmlc/RTS_Data/SourceData) and create a `System` using
[PowerSystems.jl](github.com/NREL-SIIP/PowerSystems.jl)

### Environment
This notebook depends on the SIIPExamples.jl environment which is loaded by default

In [1]:
using Pkg
Pkg.status()

Project SIIPExamples v0.0.1
Status `~/Documents/repos/SIIPExamples.jl/Project.toml`
  [336ed68f] CSV v0.7.4
  [9961bab8] Cbc v0.7.0
  [41994980] D3TypeTrees v0.1.1
  [a93c6f00] DataFrames v0.21.4
  [becb17da] Feather v0.5.6
  [2cd47ed4] InfrastructureSystems v0.13.0 [`../InfrastructureSystems.jl`]
  [b6b21f68] Ipopt v0.6.2
  [2535ab7d] JSON2 v0.3.2
  [4076af6c] JuMP v0.21.3
  [98b081ad] Literate v2.5.0
  [47be7bcc] ORCA v0.4.0
  [f0f68f2c] PlotlyJS v0.13.1
  [91a5bcdd] Plots v1.5.5
  [5f7eddb3] PowerGraphics v0.4.2 [`../PowerGraphics.jl`]
  [e690365d] PowerSimulations v0.6.0 [`../PowerSimulations.jl`]
  [bcd98974] PowerSystems v0.23.2 [`../PowerSystems.jl`]
  [9e3dc215] TimeSeries v0.18.0
  [f269a46b] TimeZones v1.3.1
  [0f1e0344] WebIO v0.8.14
  [ade2ca70] Dates 
  [56ddb016] Logging 
  [44cfe95a] Pkg 
  [9a3f8284] Random 
  [10745b16] Statistics 
  [8dfed614] Test 


### Dependencies

In [2]:
using SIIPExamples
using PowerSystems
using TimeSeries
using Dates

### Fetch Data
PowerSystems.jl links to some test data that is suitable for this example.
Let's download the test data

In [3]:
PowerSystems.download(PowerSystems.TestData; branch = "master") # *note* add `force=true` to get a fresh copy
base_dir = dirname(dirname(pathof(PowerSystems)));

### The tabular data format relies on a folder containing `*.csv` files and a `user_descriptors.yaml` file
First, we'll read the tabular data

In [4]:
RTS_GMLC_DIR = joinpath(base_dir, "data", "RTS_GMLC");
rawsys = PowerSystems.PowerSystemTableData(
    RTS_GMLC_DIR,
    100.0,
    joinpath(RTS_GMLC_DIR, "user_descriptors.yaml"),
    timeseries_metadata_file = joinpath(RTS_GMLC_DIR, "timeseries_pointers.json"),
    generator_mapping_file = joinpath(RTS_GMLC_DIR, "generator_mapping.yaml"),
)

┌ Info: Parsing csv files in Reserves ...
└ @ PowerSystems /Users/cbarrows/Documents/repos/PowerSystems.jl/src/parsers/power_system_table_data.jl:125
┌ Info: Parsing csv data in DAY_AHEAD_regional_Flex_Down.csv ...
└ @ PowerSystems /Users/cbarrows/Documents/repos/PowerSystems.jl/src/parsers/power_system_table_data.jl:129
┌ Info: Parsing csv data in DAY_AHEAD_regional_Flex_Up.csv ...
└ @ PowerSystems /Users/cbarrows/Documents/repos/PowerSystems.jl/src/parsers/power_system_table_data.jl:129
┌ Info: Parsing csv data in DAY_AHEAD_regional_Reg_Down.csv ...
└ @ PowerSystems /Users/cbarrows/Documents/repos/PowerSystems.jl/src/parsers/power_system_table_data.jl:129
┌ Info: Parsing csv data in DAY_AHEAD_regional_Reg_Up.csv ...
└ @ PowerSystems /Users/cbarrows/Documents/repos/PowerSystems.jl/src/parsers/power_system_table_data.jl:129
┌ Info: Parsing csv data in DAY_AHEAD_regional_Spin_Up_R1.csv ...
└ @ PowerSystems /Users/cbarrows/Documents/repos/PowerSystems.jl/src/parsers/power_system_table_da

PowerSystems.PowerSystemTableData:
  directory:  /Users/cbarrows/Documents/repos/PowerSystems.jl/data/RTS_GMLC
  timeseries_metadata_file:  /Users/cbarrows/Documents/repos/PowerSystems.jl/data/RTS_GMLC/timeseries_pointers.json
  base_power:  100.0
  BRANCH:  120×14 DataFrame
  BUS:  73×15 DataFrame
  GENERATOR:  158×58 DataFrame
  RESERVE:  7×7 DataFrame
  DC_BRANCH:  1×60 DataFrame
  STORAGE:  22×7 DataFrame


### Create a `System`
Next, we'll create a `System` from the `rawsys` data. Since a `System` is predicated on a
forecast resolution and the `rawsys` data includes both 5-minute and 1-hour resolution
forecasts, we also need to specify which forecasts we want to include in the `System`.
The `forecast_resolution` kwarg filters to only include forecasts with a matching resolution.

In [5]:
sys = System(rawsys; forecast_resolution = Dates.Hour(1));
sys

┌ Warning: Missing LOAD data.
└ @ PowerSystems /Users/cbarrows/Documents/repos/PowerSystems.jl/src/parsers/power_system_table_data.jl:212
┌ Warning: User-defined column name Startup Ramp Rate MW/min is not in dataframe.
└ @ PowerSystems /Users/cbarrows/Documents/repos/PowerSystems.jl/src/parsers/power_system_table_data.jl:1196
┌ Warning: User-defined column name Shutdown Ramp Rate MW/min is not in dataframe.
└ @ PowerSystems /Users/cbarrows/Documents/repos/PowerSystems.jl/src/parsers/power_system_table_data.jl:1196
┌ Warning: User-defined column name Status at Start is not in dataframe.
└ @ PowerSystems /Users/cbarrows/Documents/repos/PowerSystems.jl/src/parsers/power_system_table_data.jl:1196
┌ Warning: User-defined column name Time at Status is not in dataframe.
└ @ PowerSystems /Users/cbarrows/Documents/repos/PowerSystems.jl/src/parsers/power_system_table_data.jl:1196
┌ Warning: User-defined column name Start Cost Cold is not in dataframe.
└ @ PowerSystems /Users/cbarrows/Documents/

System
======
Base Power: 100.0

Components
==========
Num components: 434

15×3 DataFrame. Omitted printing of 2 columns
│ Row │ ConcreteType                              │
│     │ String                                    │
├─────┼───────────────────────────────────────────┤
│ 1   │ Area                                      │
│ 2   │ Bus                                       │
│ 3   │ GenericBattery                            │
│ 4   │ HVDCLine                                  │
│ 5   │ HydroDispatch                             │
│ 6   │ HydroEnergyReservoir                      │
│ 7   │ Line                                      │
│ 8   │ LoadZone                                  │
│ 9   │ PowerLoad                                 │
│ 10  │ RenewableDispatch                         │
│ 11  │ RenewableFix                              │
│ 12  │ TapTransformer                            │
│ 13  │ ThermalStandard                           │
│ 14  │ VariableReserve{PowerSystems.ReserveDown} │
│ 15  │ VariableReserve{PowerSystems.ReserveUp}   │

Forecasts
=========
Resolution: 60 minutes
Horizon: 24
Initial Times: 2020-01-01T00:00:00
Interval: 0 seconds
Components with Forecasts: 139
Total Forecasts: 179

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*